In [2]:
import json
import torch
import numpy as np
from transformers import AutoTokenizer, AutoModel
from tqdm import tqdm

In [3]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')
model = AutoModel.from_pretrained('bert-base-cased')
id2name = json.load(open('datasets/MovieLens-1M-5Star/id2all.json'))

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
embs = []
for name in tqdm(id2name):
    inputs = tokenizer(name, return_tensors='pt')
    outputs = model(**inputs)
    embs.append(outputs.pooler_output.detach().numpy()[0])
embs = np.array(embs)
np.save('embs_bert_base_cased_(title+synoposis).npy', embs)

100%|██████████| 3645/3645 [03:36<00:00, 16.81it/s]


In [6]:
lens = []
for name in tqdm(id2name):
    inputs = tokenizer(name)['input_ids']
    lens.append(len(inputs))
print(np.mean(lens), np.std(lens))

100%|██████████| 3645/3645 [00:01<00:00, 3310.24it/s]

72.66227709190672 37.96141013301181


In [7]:
# embs = np.load('embs_bert_base_cased.npy')
embs = embs / np.linalg.norm(embs, ord=2, axis=-1, keepdims=True)
sims = embs @ embs.T

In [8]:
f1 = open('emb_projector/vectors.tsv', 'w')
f2 = open('emb_projector/metadata.tsv', 'w')

for emb, name in zip(embs, id2name):
    f1.write('\t'.join(map(str, emb.tolist())) + '\n')
    f2.write(name + '\n')

f1.close()
f2.close()

In [9]:
idx = 2
print(id2name[idx])
print()
for sim_idx in (-sims[idx]).argsort()[:20]:
    print(id2name[sim_idx], sims[idx][sim_idx])


Grumpier Old Men (A family wedding reignites the ancient feud between next-door neighbors and fishing buddies John and Max. Meanwhile, a sultry Italian divorcée opens a restaurant at the local bait shop, alarming the locals who worry she'll scare the fish away. But she's less interested in seafood than she is in cooking up a hot time with Max.)

Grumpier Old Men (A family wedding reignites the ancient feud between next-door neighbors and fishing buddies John and Max. Meanwhile, a sultry Italian divorcée opens a restaurant at the local bait shop, alarming the locals who worry she'll scare the fish away. But she's less interested in seafood than she is in cooking up a hot time with Max.) 1.0
My Bodyguard (Clifford Peach, an easygoing teenager, is finding less than easy to fit in at his new high school, where a tough-talking bully terrorizes his classmates and extorts their lunch money. Refusing to pay up, Clifford enlist the aid of an overgrown misfit whose mere presence intimidates stud

In [15]:
idx = 2
print(id2name[idx])
print()
for sim_idx in (-sims[idx]).argsort()[:20]:
    print(id2name[sim_idx], sims[idx][sim_idx])


Grumpier Old Men

Grumpier Old Men 1.0
Grumpy Old Men 0.998638
The Kid 0.9954133
The Kid 0.9954133
Bushwhacked 0.99519026
The Man with the Golden Gun 0.9951815
The Man with the Golden Arm 0.99513876
A Few Good Men 0.9951285
The Suburbans 0.9949783
Things Change 0.9949132
Honey, I Blew Up the Kid 0.9948431
Bad Company 0.994802
The Man Who Knew Too Little 0.99469674
The American President 0.9946568
Somebody is Waiting 0.994637
A Fistful of Dollars 0.9946312
The Wrong Trousers 0.99462044
Love's Labour's Lost 0.99459875
The Absent Minded Professor 0.9945713
The Bear 0.99453956


In [39]:
movie_data = json.load(open('tmdb_movie_info.json'))
len(movie_data)
movie_data[4]['overview']
with open('movie_data.tsv', 'w') as fout:
    for item in movie_data:
        title = ' '.join(item['title'].strip().split())
        overview = ' '.join(item['overview'].strip().split())
        if len(title) == 0 or len(overview) == 0: continue
        fout.write(title + '\t' + overview + '\n')